# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [87]:
sc

Calculation started (calculation_id=16c5a340-1cc2-4286-3b8f-3abc485e14cc) in (session=8cc5a30b-da87-6b13-6287-b27c61b4cc1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [88]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=f0c5a340-1dea-32e5-8f9e-90e4105a25c9) in (session=8cc5a30b-da87-6b13-6287-b27c61b4cc1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [89]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

Calculation started (calculation_id=f4c5a340-1f57-6113-28cd-48fd4c0a5615) in (session=8cc5a30b-da87-6b13-6287-b27c61b4cc1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [90]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'nchi7'

Calculation started (calculation_id=12c5a340-2078-9539-eca1-432f64e96008) in (session=8cc5a30b-da87-6b13-6287-b27c61b4cc1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [91]:
#export
def long_trips(trips):
    
    # Returns a Dataframe (trips) with Schema the same as :trips:
    
    from pyspark.sql.functions import col
    trips = trips.filter(col("trip_distance") >= 2)
    
    return trips

Calculation started (calculation_id=1ec5a340-21d2-5468-1542-e0bfaf93d92d) in (session=8cc5a30b-da87-6b13-6287-b27c61b4cc1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [92]:
#export
def manhattan_trips(trips, lookup):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, sum
    from pyspark.sql.window import Window
    from pyspark.sql import functions as F
    
    # Filter trips with DOLocationID in Manhattan
    manhattan_trips_df = trips.join(lookup, trips.DOLocationID == lookup.LocationID, "inner").filter(lookup.Borough == "Manhattan")

    # Group by DOLocationID and sum passenger count
    manhattan_trip_agg = manhattan_trips_df.groupBy("DOLocationID").agg(sum("passenger_count").alias("pcount"))

    # Rank the locations by passenger count in descending order
    windowSpec = Window.orderBy(F.desc("pcount"))
    manhattan_trip_agg = manhattan_trip_agg.withColumn("rank", F.rank().over(windowSpec))

    # Select the top 20 locations
    mtrips = manhattan_trip_agg.filter(col("rank") <= 20).drop("rank")
    
    return mtrips

Calculation started (calculation_id=a0c5a340-2325-9f85-a2af-cf7baa615e02) in (session=8cc5a30b-da87-6b13-6287-b27c61b4cc1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [93]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    from pyspark.sql.functions import col, avg, count, when, lit
    from pyspark.sql.types import DecimalType
    
    mtrips = mtrips.withColumnRenamed("DOLocationID", "M_DOLocationID")
    
    wp = trips.join(mtrips, trips['DOLocationID'] == mtrips['M_DOLocationID'], "left")
    
    # Calculate the weighted profit for each PULocationID
    wp = wp.groupBy("PULocationID").agg(avg("total_amount").alias("avg_total_amount"),count("*").alias("total_trips"),
        sum(when(col("M_DOLocationID").isNotNull(), 1)).alias("trips_to_top20"))

    # Calculate the weighted profit using the formula
    wp = wp.withColumn("weighted_profit",(col("trips_to_top20") / col("total_trips"))*col("avg_total_amount").cast(DecimalType(38, 18)))
    
    # Select only the relevant columns
    wp = wp.select("PULocationID", "weighted_profit")
    
    return wp

Calculation started (calculation_id=60c5a340-2476-5fe6-6b1d-fd4cc5e78493) in (session=8cc5a30b-da87-6b13-6287-b27c61b4cc1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [94]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    from pyspark.sql import functions as F
    from pyspark.sql.types import StructType, StructField, StringType, DecimalType
    # Join the wp (weighted_profit) DataFrame with the lookup DataFrame to get zone and borough information
    final = wp.join(lookup, wp.PULocationID == lookup.LocationID, "inner").select("Zone", "Borough", "weighted_profit")

    # Rank the locations by weighted profit in descending order
    windowSpec = Window.orderBy(F.desc("weighted_profit"))
    final = final.withColumn("rank", F.rank().over(windowSpec))

    # Select the top 20 locations with the highest weighted profit
    final = final.filter(col("rank") <= 20).drop("rank")  
    
    from pyspark.sql.types import DecimalType
    final = final.select(final["Zone"].cast(StringType()).alias("Zone"),final["Borough"].cast(StringType()).alias("Borough"), final["weighted_profit"].cast(DecimalType(38, 10)).alias("weighted_profit"))
    
    return final

Calculation started (calculation_id=36c5a340-2587-5666-cb1a-3a6f97f6103a) in (session=8cc5a30b-da87-6b13-6287-b27c61b4cc1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [95]:
main(size='large', bucket = 's3://cse6242-nchi7/q3-output-large12')

Calculation started (calculation_id=2cc5a340-26aa-8353-5c78-2f5b6c44e2b3) in (session=8cc5a30b-da87-6b13-6287-b27c61b4cc1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
nchi7
Trip Count:  187203269
+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|        Baisley Park|       Queens|  29.3604557791|
|Flushing Meadows-...|       Queens|  27.3048457336|
|       South Jamaica|       Queens|  26.2949162399|
|     Randalls Island|    Manhattan|  24.1509899402|
|        Astoria Park|       Queens|  21.7064171121|
|Briarwood/Jamaica...|       Queens|  19.9450646318|
|Springfield Garde...|       Queens|  19.4683092888|
|             Jamaica|       Queens|  19.2839430001|
|              Corona|       Queens|  18.2287692482|
|   LaGuardia Airport|       Queens|  18.1813388084|
|         Jamaica Bay|       Queens|  17.1005294468|
|             Maspeth|       Queens|  17.0054506401|
|Eltingville/Annad...|Staten Island|  16.8377647569|
|         JFK Airport|       Queens|  16.7777253482|
|        Battery Park|    Manhattan|  12.849780

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

trips.show(5)

manhattan_trips(trips, lookup).show()

weighted_profit(trips,mtrips).show()

final_output(wp, lookup).show()